# Solution for Assignment 3: Newsvendor Model

This notebook solves the problems from Assignment 3 using the `imf_*` formula library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from scipy.stats import norm

# Get the absolute path of the current file and add parent directory to path
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Now we can import from imf_main
try:
    from imf_main import (
        newsvendor_critical_ratio,
        newsvendor_normal,
        NormalDistribution,
        expected_shortage
    )
except ImportError:
    print("Could not import from imf_main. Using direct implementation...")
    
    # Direct implementation of newsvendor functions if import fails
    def newsvendor_critical_ratio(price, cost, salvage=0):
        """Calculate newsvendor critical ratio"""
        return (price - cost) / (price - salvage)
    
    def newsvendor_normal(mean, std, critical_ratio):
        """Calculate newsvendor order quantity for normally distributed demand"""
        z = norm.ppf(critical_ratio)
        return mean + z * std
    
    class NormalDistribution:
        @staticmethod
        def pdf(z, label="", suffix=""):
            return norm.pdf(z)
        
        @staticmethod
        def cdf(z, label="", suffix=""):
            return norm.cdf(z)
        
        @staticmethod
        def loss_function(z, label="", suffix=""):
            return norm.pdf(z) - z * (1 - norm.cdf(z))
        
    def expected_shortage(sigma, z, label="", suffix=""):
        """Calculate expected shortage"""
        return sigma * (norm.pdf(z) - z * (1 - norm.cdf(z)))

## Exercise 3(a): Basic Newsvendor Model

In [ ]:
# Given data
mean_demand = 2000  # units
std_demand = 250  # units
price = 6  # euro
cost = 3  # euro
salvage = 2.5  # euro

# Calculate critical ratio using the library function
critical_ratio = newsvendor_critical_ratio(price, cost, salvage)
print(f"Critical ratio: {critical_ratio:.4f}")

# Calculate optimal order quantity using the library function
optimal_q = newsvendor_normal(mean_demand, std_demand, critical_ratio)
print(f"Newsvendor quantity: {optimal_q:.2f} units")

## Exercise 3(b): KPI calculations

In [ ]:
# Calculate z-value
z = (optimal_q - mean_demand) / std_demand
print(f"Safety factor z: {z:.4f}")

# Calculate expected lost sales
els = expected_shortage(std_demand, z)
print(f"Expected lost sales: {els:.2f} units")

# Calculate expected sales
expected_sales = mean_demand - els
print(f"Expected sales: {expected_sales:.2f} units")

# Calculate expected leftover
expected_leftover = optimal_q - expected_sales
print(f"Expected leftover: {expected_leftover:.2f} units")

# Calculate expected profit
expected_profit = -cost * optimal_q + price * expected_sales + salvage * expected_leftover
print(f"Expected profit: {expected_profit:.2f} euro")

# Calculate service level metrics
service_level_availability = critical_ratio  # alpha = beta
print(f"Service level (Availability): {service_level_availability:.4f}")

fill_rate = expected_sales / mean_demand
print(f"Service level (Fill-rate): {fill_rate:.4f}")

## Alternative Method Using Direct Calculations

In [ ]:
# Calculate critical ratio directly
beta = (price - cost) / (price - salvage)
print(f"Critical ratio (direct calculation): {beta:.4f}")

# Calculate optimal order quantity
z_value = norm.ppf(beta)  # Inverse CDF (percentile) of the standard normal
qt = mean_demand + std_demand * z_value
print(f"Newsvendor quantity (direct calculation): {qt:.2f} units")

# Calculate expected lost sales using normal loss function
def normal_loss_function(z):
    return norm.pdf(z) - z * (1 - norm.cdf(z))

ELS = std_demand * normal_loss_function(z_value)
print(f"Expected lost sales (direct calculation): {ELS:.2f} units")

# Calculate other KPIs
ES = mean_demand - ELS
print(f"Expected sales (direct calculation): {ES:.2f} units")

ELO = qt - ES
print(f"Expected leftover (direct calculation): {ELO:.2f} units")

EP = -cost * qt + price * ES + salvage * ELO
print(f"Expected profit (direct calculation): {EP:.2f} euro")

alpha = beta
print(f"Service level availability (direct calculation): {alpha:.4f}")

fill_rate = ES / mean_demand
print(f"Service level fill-rate (direct calculation): {fill_rate:.4f}")

## Visualization of Newsvendor Model Solution

In [ ]:
# Create range of demand values for plotting
demand_values = np.linspace(mean_demand - 4 * std_demand, mean_demand + 4 * std_demand, 1000)

# Calculate normal PDF values using scipy
pdf_values = norm.pdf(demand_values, loc=mean_demand, scale=std_demand)

# Create plot
plt.figure(figsize=(12, 6))

# Plot demand distribution
plt.subplot(1, 2, 1)
plt.plot(demand_values, pdf_values)
plt.fill_between(demand_values[demand_values <= optimal_q], pdf_values[demand_values <= optimal_q], 
                alpha=0.3, color='green', label='No stockout')
plt.fill_between(demand_values[demand_values > optimal_q], pdf_values[demand_values > optimal_q], 
                alpha=0.3, color='red', label='Stockout')

# Mark key points
plt.axvline(x=mean_demand, color='blue', linestyle='--', label='Mean demand')
plt.axvline(x=optimal_q, color='green', linestyle='-', label='Optimal order quantity')

# Add labels and legend
plt.xlabel('Demand')
plt.ylabel('Probability Density')
plt.title(f'Newsvendor Solution (CR = {critical_ratio:.4f})')
plt.grid(True, alpha=0.3)
plt.legend()

# Create a cost/profit plot
plt.subplot(1, 2, 2)

# Define range of order quantities
q_values = np.linspace(mean_demand - 2 * std_demand, mean_demand + 2 * std_demand, 100)

# Calculate expected profit for different order quantities
profits = []
for q in q_values:
    z = (q - mean_demand) / std_demand
    els = std_demand * normal_loss_function(z)
    exp_sales = mean_demand - els
    exp_leftover = q - exp_sales
    profit = -cost * q + price * exp_sales + salvage * exp_leftover
    profits.append(profit)

plt.plot(q_values, profits)
plt.axvline(x=optimal_q, color='green', linestyle='-', label='Optimal order quantity')
plt.axhline(y=max(profits), color='red', linestyle='--', label='Maximum expected profit')

# Add labels and legend
plt.xlabel('Order Quantity')
plt.ylabel('Expected Profit')
plt.title('Expected Profit vs Order Quantity')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

## Critical Ratio Analysis

Let's explore how the critical ratio and optimal order quantity change as we vary the price, cost, and salvage value.

In [ ]:
# Define ranges for parameters
prices = np.linspace(4, 10, 20)  # Varying selling price
costs = [cost]  # Fixed cost
salvages = [salvage]  # Fixed salvage value

# Calculate critical ratios and optimal quantities for different prices
cr_values = []
q_values = []

for p in prices:
    cr = newsvendor_critical_ratio(p, cost, salvage)
    cr_values.append(cr)
    q_values.append(newsvendor_normal(mean_demand, std_demand, cr))

# Plot results
plt.figure(figsize=(12, 5))

# Plot critical ratio vs price
plt.subplot(1, 2, 1)
plt.plot(prices, cr_values)
plt.axhline(y=0.5, color='r', linestyle='--')
plt.axhline(y=0.95, color='r', linestyle=':')
plt.axhline(y=0.05, color='r', linestyle=':')
plt.xlabel('Selling Price')
plt.ylabel('Critical Ratio')
plt.title('Critical Ratio vs Selling Price')
plt.grid(True, alpha=0.3)

# Plot optimal quantity vs price
plt.subplot(1, 2, 2)
plt.plot(prices, q_values)
plt.axhline(y=mean_demand, color='b', linestyle='--', label='Mean Demand')
plt.xlabel('Selling Price')
plt.ylabel('Optimal Order Quantity')
plt.title('Optimal Order Quantity vs Selling Price')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

## Sensitivity Analysis of Optimal Order Quantity to Demand Uncertainty

In [ ]:
# Define range of standard deviations
std_values = np.linspace(50, 500, 20)

# Calculate optimal quantities for different standard deviations
q_values = []
for std in std_values:
    q_values.append(newsvendor_normal(mean_demand, std, critical_ratio))

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(std_values, q_values, 'b-')
plt.axhline(y=mean_demand, color='r', linestyle='--', label='Mean Demand')
plt.xlabel('Demand Standard Deviation')
plt.ylabel('Optimal Order Quantity')
plt.title('Sensitivity of Optimal Order Quantity to Demand Uncertainty')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()